<a href="https://colab.research.google.com/github/drago467/Generative-AI-networks/blob/main/Advanced_Generative_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Advanced GAN
# https://medium.com/@ideami

In [2]:
#importing the libraries
import torch, torchvision, os, PIL, pdb
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
from tqdm.auto import tqdm
from PIL import Image
import matplotlib.pyplot as plt

def show(tensor, num = 25, wanb = 0, name = ''):
  data = tensor.detach().cpu()
  grid = make_grid(data[:num], nrow=5).permute(1, 2, 0)

### hyperparameters and general parameters
n_epochs = 10000
batch_size = 128
lr = 1e-4
z_dim = 200
device = 'cuda' #GPU

cur_step = 0
crit_cyrcles = 5
gen_losses = []
crit_losses = []
show_step = 35
save_step = 35

wandbact = 1 #yes, we want to track stats through weights and biases, optinal
